In [3]:
# 필요 라이브러리 import
import os
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

plt.style.use('default')
pd.options.display.max_columns = 100

# 데이터셋 경로 설정
# filtering_labels.py에서 생성한 filtered_labels 경로 설정
filtered_train = "../filtered_labels/training_label"
filtered_val = "../filtered_labels/validation_label"

#### json 파일의 형태 확인하기


In [24]:
# 표에 이미 주어져있긴 하다
import os

file_path = r'D:\데크캠\filtered_labels\training_label\W_71920_60_mods_M_233674.json'


# JSON 파일 읽기
with open(file_path, 'r') as file:
    data = json.load(file)

# 최상위 키 출력
print("최상위 키:")
for key, value in data.items():
    print(f"키: {key}, 값 타입: {type(value)}")

print("\nitem 딕셔너리 내부:")
for key, value in data['item'].items():
    print(f"  키: {key}, 값 타입: {type(value)}")

print("\nuser 딕셔너리 내부:")
for key, value in data['user'].items():
    print(f"  키: {key}, 값 타입: {type(value)}")

# item과 user 내부의 복잡한 구조 탐색
def explore_dict(d, indent=0):
    for key, value in d.items():
        print("  " * indent + f"{key}: {type(value)}")
        if isinstance(value, dict):
            explore_dict(value, indent + 1)
        elif isinstance(value, list) and len(value) > 0:
            print("  " * (indent + 1) + f"리스트의 첫 번째 항목 타입: {type(value[0])}")

print("\nitem의 상세 구조:")
explore_dict(data['item'])

print("\nuser의 상세 구조:")
explore_dict(data['user'])

최상위 키:
키: E_id, 값 타입: <class 'int'>
키: imgName, 값 타입: <class 'str'>
키: item, 값 타입: <class 'dict'>
키: user, 값 타입: <class 'dict'>

item 딕셔너리 내부:
  키: imgName, 값 타입: <class 'str'>
  키: era, 값 타입: <class 'int'>
  키: style, 값 타입: <class 'str'>
  키: gender, 값 타입: <class 'str'>
  키: survey, 값 타입: <class 'dict'>

user 딕셔너리 내부:
  키: R_id, 값 타입: <class 'int'>
  키: r_gender, 값 타입: <class 'int'>
  키: age, 값 타입: <class 'int'>
  키: mar, 값 타입: <class 'int'>
  키: job, 값 타입: <class 'int'>
  키: income, 값 타입: <class 'int'>
  키: r_style1, 값 타입: <class 'int'>
  키: r_style2, 값 타입: <class 'int'>
  키: r_style3, 값 타입: <class 'int'>
  키: r_style4, 값 타입: <class 'int'>
  키: r_style5, 값 타입: <class 'int'>

item의 상세 구조:
imgName: <class 'str'>
era: <class 'int'>
style: <class 'str'>
gender: <class 'str'>
survey: <class 'dict'>
  Q1: <class 'int'>
  Q2: <class 'int'>
  Q3: <class 'int'>
  Q411: <class 'int'>
  Q412: <class 'int'>
  Q413: <class 'int'>
  Q414: <class 'int'>
  Q4201: <class 'int'>
  Q4202: <class 'int'>

In [29]:
import os
import pandas as pd
import json

# JSON 파일들이 있는 폴더 경로 설정
folder_path1 = r'D:\데크캠\filtered_labels\training_label'
folder_path2 = r'D:\데크캠\filtered_labels\validation_label'

In [60]:
import os
import pandas as pd
import json

# json 파일 내 필요한 정보만 추출하는 함수 설정

def preference(label_me, output_file):
    # JSON 파일들이 있는 폴더 경로 설정
    folder_path = label_me

    # 데이터프레임을 만들기 위한 리스트 초기화
    records = []

    # 폴더 내 모든 JSON 파일 처리
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)

            # JSON 파일 로드
            with open(file_path, 'r') as f:
                try:
                    data = json.load(f)

                    # 응답자 정보 (user 딕셔너리)
                    r_id = data['user']['R_id']

                    # item 딕셔너리에서 imgName과 Q5 값 추출
                    img_name = data['item']['imgName']
                    q5_value = data['item']['survey']['Q5']

                    # Q5 값에 따라 선호와 비선호 분류
                    if q5_value == 2:
                        preference = img_name  # 스타일 선호
                        records.append({'응답자 ID': r_id, '스타일 선호': preference, '스타일 비선호': None})
                    elif q5_value == 1:
                        preference = img_name  # 스타일 비선호
                        records.append({'응답자 ID': r_id, '스타일 선호': None, '스타일 비선호': preference})

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_name}: {e}")
                except KeyError as e:
                    print(f"Missing key {e} in file {file_name}")

    # 데이터프레임 생성
    df = pd.DataFrame(records)

    # 결측치를 전방 및 후방 채우기 (fillna)
    df['스타일 선호'] = df['스타일 선호'].fillna(method='ffill')
    df['스타일 비선호'] = df['스타일 비선호'].fillna(method='bfill')

    # 최종적으로 응답자 ID 기준으로 그룹화하여 스타일 선호와 비선호를 분리
    final_records = []
    for r_id in df['응답자 ID'].unique():
        grouped = df[df['응답자 ID'] == r_id]
        
        for index, row in grouped.iterrows():
            final_records.append({
                '응답자 ID': r_id,
                '스타일 선호': row['스타일 선호'],
                '스타일 비선호': row['스타일 비선호']
            })

    # 최종 데이터프레임 생성
    final_df = pd.DataFrame(final_records)

    # 원하는 파일로 저장 (CSV 파일)
    final_df.to_csv(output_file, encoding='utf-8-sig', index=False)

    print(f"데이터가 {output_file}에 저장되었습니다.")
    
    return final_df  # 최종 데이터프레임 반환


In [61]:
# 함수 사용
df_train = preference(folder_path1,'train_preference.csv')
df_val = preference(folder_path2,'val_preference.csv')

C:\Users\82108\AppData\Local\Temp\ipykernel_2308\3429570436.py:46: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['스타일 선호'] = df['스타일 선호'].fillna(method='ffill')
C:\Users\82108\AppData\Local\Temp\ipykernel_2308\3429570436.py:47: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['스타일 비선호'] = df['스타일 비선호'].fillna(method='bfill')


데이터가 train_preference.csv에 저장되었습니다.
데이터가 val_preference.csv에 저장되었습니다.


In [67]:
# csv 불러와서 사용
df_train = pd.read_csv('train_preference.csv')
df_val = pd.read_csv('val_preference.csv')


In [71]:
# df_train과 df_valid을 합치는데 '응답자 ID'를 기준으로 합친다., 열의 값은 outer로 설정한다, 응답자 id를 행으로 설정한다
df = pd.merge(df_train, df_val, on='응답자 ID', how='outer', suffixes=('_train', '_val')).set_index('응답자 ID')

In [72]:
df.head(10)

,스타일 선호_train,스타일 비선호_train,스타일 선호_val,스타일 비선호_val
응답자 ID,,,,
12,W_03412_50_classic_W.jpg,W_03412_50_classic_W.jpg,W_02557_00_ecology_W.jpg,W_02651_50_feminine_W.jpg
12,W_03412_50_classic_W.jpg,W_03412_50_classic_W.jpg,W_03412_50_classic_W.jpg,W_03412_50_classic_W.jpg
25,T_01475_10_sportivecasual_M.jpg,T_01477_19_normcore_M.jpg,NaN,NaN
25,T_14304_60_mods_M.jpg,T_14461_19_normcore_W.jpg,NaN,NaN
25,T_16423_10_sportivecasual_M.jpg,T_16423_10_sportivecasual_M.jpg,NaN,NaN
25,W_12723_10_sportivecasual_M.jpg,W_12740_00_metrosexual_M.jpg,NaN,NaN
26,T_09447_19_lounge_W.jpg,T_09449_60_minimal_W.jpg,NaN,NaN
26,W_18986_90_kitsch_W.jpg,W_18990_50_feminine_W.jpg,NaN,NaN
27,W_03007_70_hippie_M.jpg,W_03007_70_hippie_M.jpg,W_06522_50_ivy_M.jpg,W_06522_50_ivy_M.jpg


# 100명 응답자를 추출해내는건가? NaN값은 어떻게 처리 해야하지?